In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.svm import SVC

In [2]:
df = pd.read_csv("BankChurners.csv").drop(columns = ['CLIENTNUM'])
bank_data = df

In [73]:
#Transfer card category to integers, because there is a hierarical relationship
card_cate_map = {'Blue': 0, 'Silver': 1, 'Gold':2, 'Platinum': 3}
for i in range(len(bank_data['Card_Category'])):
    for j in card_cate_map:
        if bank_data['Card_Category'][i] == j:
            bank_data.at[i, 'Card_Category'] = card_cate_map[j]       

#Transfer education category to integers, because there is a hierarical relationship
education_cate_map = {'Unknown':-1,'Uneducated':0,'High School':1,'College':2,'Graduate':3,'Post-Graduate':4, 'Doctorate':5 }
for i in range(len(bank_data['Education_Level'])):
    for j in education_cate_map:
        if bank_data['Education_Level'][i] == j:
            bank_data.at[i, 'Education_Level'] = education_cate_map[j]

income_cate_map = {'Unknown': -1, 'Less than $40K': 0, '$40K - $60K': 1, '$60K - $80K': 2, '$80K - $120K':3, '$120K +': 4}
for i in range(len(bank_data['Income_Category'])):
    for j in income_cate_map:
        if bank_data['Income_Category'][i] == j:
            bank_data.at[i, 'Income_Category'] = income_cate_map[j]
            
attrition_flag_map = {'Existing Customer': '0', 'Attrited Customer': '1'}
for i in range(len(bank_data['Attrition_Flag'])):
    for j in attrition_flag_map:
        if bank_data['Attrition_Flag'][i] == j:
            bank_data.at[i, 'Attrition_Flag'] = attrition_flag_map[j]

In [74]:
data = pd.get_dummies(bank_data, columns = ['Gender','Marital_Status'])
data = data.dropna()
data

,Attrition_Flag,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,...,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Gender_F,Gender_M,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_Unknown
0,0,45,3,1,2,0,39,5,1,3,...,1144,42,1.625,0.061,0,1,0,1,0,0
1,0,49,5,3,0,0,44,6,1,2,...,1291,33,3.714,0.105,1,0,0,0,1,0
2,0,51,3,3,3,0,36,4,1,0,...,1887,20,2.333,0.000,0,1,0,1,0,0
3,0,40,4,1,0,0,34,3,4,1,...,1171,20,2.333,0.760,1,0,0,0,0,1
4,0,40,3,0,2,0,21,5,1,0,...,816,28,2.500,0.000,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,0,50,2,3,1,0,40,3,2,3,...,15476,117,0.857,0.462,0,1,0,0,1,0
10123,1,41,2,-1,1,0,25,4,2,3,...,8764,69,0.683,0.511,0,1,1,0,0,0
10124,1,44,1,1,0,0,36,5,3,4,...,10291,60,0.818,0.000,1,0,0,1,0,0
10125,1,30,2,3,1,0,36,4,3,3,...,8395,62,0.722,0.000,0,1,0,0,0,1


In [75]:
data = data.to_numpy()

In [77]:
X = data[:, 1:]
y = data[:, 0]

len([i for i in y if i == 1])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

last element of y: 1


1171

In [56]:
# define the scoring method
scoring = 'accuracy'

# define models to train
names = ['Decision Tree', 'SVM Linear', 'SVM RBF', 'SVM Sigmoid']

# build classifiers
classifiers = [
    tree.DecisionTreeClassifier(),
    SVC(kernel='linear', gamma='auto'),
    SVC(kernel='rbf', gamma='auto'),
    SVC(kernel='sigmoid', gamma='auto'),
]

models = zip(names, classifiers)

# evaluate each model in turn
results_1 = []
#results_2 = []
#results_3 = []
names = []

for name, model in models:
    ## Use stratified k-fold cross validation to deal with imbalanced data
    kfold = StratifiedKFold(n_splits=5, shuffle=True)
    for train_index, test_index in kfold.split(X, y):
        print("TRAIN:", train_index, "TEST:", test_index)
#     cv_results_1 = cross_val_score(model, X_train_1, y_train_1, cv=kfold, scoring=scoring)
#     #cv_results_2 = cross_val_score(model, X_train_2, y_train_2, cv=kfold, scoring=scoring)
#     #cv_results_3 = cross_val_score(model, X_train_3, y_train_3, cv=kfold, scoring=scoring)
#     results_1.append(cv_results_1)
#     #results_2.append(cv_results_2)
#     #results_3.append(cv_results_3)
#     names.append(name)
#     #msg = '{0}:\nPreprocess 1: {1}  ({2})\nPreprocess 2: {3}  ({4})\nPreprocess 3: {5}  ({6})'.format(name, cv_results_1.mean(), cv_results_1.std(), cv_results_2.mean(), cv_results_2.std(), cv_results_3.mean(), cv_results_3.std())
#     msg = '{0}:\nPreprocess 1: {1}  ({2})'.format(name, cv_results_1.mean(), cv_results_1.std())
#     print(msg)

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'unknown' instead.